<div class="alert" style="background-color:#006400; color:white; padding:0px 10px; border-radius:5px;"><h1 style='margin:15px 15px; color:#FFFFFF; font-size:32px'>HyperTuning for 180 Datasets</h1></div>

The work is under the **"Master Thesis"** by **Chau Tran** with the supervision from **Prof. Roland Olsson**.

## 1. Packages and Datasets

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, LSTM, MaxPooling2D,AveragePooling2D,GlobalMaxPooling2D, GlobalAveragePooling2D, Flatten, Dropout, Reshape, BatchNormalization, ReLU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorboard
import kerastuner as kt #(kt.tuners.RandomSearch, kt.tuners.Hyperband)
from kerastuner_tensorboard_logger import (
    TensorBoardLogger,
    setup_tb  # Optional
)
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, RobustScaler

from functools import partial
from matplotlib import rc, style
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
import pandas as pd #pd.plotting.register_matplotlib_converters
import numpy as np
from scipy import stats

import sys, os, math, time, datetime

print("kt: ", kt.__version__)
print("tf: ", tf.__version__)
print(os.getcwd())

%matplotlib inline
%config InlineBackend.figure_format='retina'

style.use("seaborn")
pd.plotting.register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale = 1)

# rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(1)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)

tf.debugging.set_log_device_placement(False)

strategy = tf.distribute.MirroredStrategy()

# Writing to file
filepath = './Version9.128timesteps/seqnetdata.ni=3.no=5.mc=15.numTimeSteps128.version9.0.csv'
with open(filepath, "r") as fp:
    [noInput, noOutput] = [int(x) for x in fp.readline().split(',')]
rdf = np.array(pd.read_csv(filepath, skiprows=1))
print(type(rdf), rdf.shape)
print(type(noInput), noInput, type(noOutput), noOutput)
display(rdf)

kt:  1.0.2
tf:  2.6.0
/home/ifeai/ChauTran/git/0_HIOF_Studying/0_MasterProject
<class 'numpy.ndarray'> (9999, 1024)
<class 'int'> 3 <class 'int'> 5


array([[0, 0, 1, ..., 1, 0, 1],
       [1, 1, 0, ..., 1, 1, 1],
       [0, 1, 0, ..., 1, 0, 1],
       ...,
       [0, 1, 1, ..., 1, 1, 1],
       [1, 0, 1, ..., 0, 1, 0],
       [1, 1, 1, ..., 1, 1, 1]])

## 2. Data Preparation

In [13]:
# np.set_printoptions(threshold=1000)
print('Step 1: Dividing the training and testing set with ratio 1:1 (50%).')
df_test, df_train = train_test_split(rdf,test_size=0.5)
print(df_train.shape, df_test.shape)

print('Step 2: Formatting the training and testing sets and generating validating set.')
# Training and Validating sets
x_train, y_train, x_val, y_val = None, None, None, None
for i in range(df_train.shape[0]):
    df_train_i = df_train[i].reshape(-1, noInput+noOutput).astype('float32')
    x_train_i, y_train_i = df_train_i[:, 0:noInput], df_train_i[:, noInput:]
#     x_train_i, x_val_i = train_test_split(x_train_i, test_size=0.078125)
#     y_train_i, y_val_i = train_test_split(y_train_i, test_size=0.078125)
    x_train = x_train_i[np.newaxis,:,:] if x_train is None else np.append(x_train, x_train_i[np.newaxis,:,:], axis=0)
    y_train = y_train_i[np.newaxis,:,:] if y_train is None else np.append(y_train, y_train_i[np.newaxis,:,:], axis=0)
#     x_val   = x_val_i[np.newaxis,:,:]   if x_val   is None else np.append(x_val, x_val_i[np.newaxis,:,:], axis=0)
#     y_val   = y_val_i[np.newaxis,:,:]   if y_val   is None else np.append(y_val, y_val_i[np.newaxis,:,:], axis=0)
print("Training set: ", x_train.shape, y_train.shape, x_train.dtype)
# print("Validating set: ", x_val.shape, y_val.shape)

#Testing set
x_test, y_test = None, None
for i in range(df_test.shape[0]):
    df_test_i = df_test[i].reshape(-1, noInput+noOutput).astype('float32')
    x_test_i, y_test_i = df_test_i[:, 0:noInput], df_test_i[:, noInput:]
    x_test = x_test_i[np.newaxis,:,:] if x_test is None else np.append(x_test, x_test_i[np.newaxis,:,:], axis=0)
    y_test = y_test_i[np.newaxis,:,:] if y_test is None else np.append(y_test, y_test_i[np.newaxis,:,:], axis=0)
print("Testing set: ", x_test.shape, y_test.shape)

Step 1: Dividing the training and testing set with ratio 1:1 (50%).
(5000, 1024) (4999, 1024)
Step 2: Formatting the training and testing sets and generating validating set.
Training set:  (5000, 128, 3) (5000, 128, 5) float32
Testing set:  (4999, 128, 3) (4999, 128, 5)


## 3. LSTM Model

In [26]:
# with strategy.scope(): 
lstm_model = tf.keras.Sequential()
lstm_model.add(
    tf.keras.layers.Bidirectional(
      tf.keras.layers.LSTM(
          units=128, 
          input_shape=[x_train.shape[1], x_train.shape[2]],
          activation='tanh', recurrent_activation='sigmoid',
          return_sequences=False
      )
    )
)
lstm_model.add(tf.keras.layers.Dropout(rate=0.5))
lstm_model.add(tf.keras.layers.Dense(units=128, activation='relu'))
lstm_model.add(tf.keras.layers.Dense(y_train.shape[1], activation='softmax'))
adam = tf.optimizers.Adam(learning_rate = 0.001, decay=1e-6)
lstm_model.compile(loss='mse', optimizer=adam, metrics=[tf.keras.metrics.MeanSquaredError()])

strt_time = datetime.datetime.now() 
lstm_model.fit(
    x_train, y_train,
    epochs=100,
    batch_size=128,
    validation_split=0.078125,
    shuffle=True,
    use_multiprocessing=True
)
curr_time = datetime.datetime.now()
display(lstm_model.summary())
timedelta = curr_time - strt_time
dnn_train_time = timedelta.total_seconds()

val_performance = lstm_model.evaluate(x_val, y_val)
performance = lstm_model.predict(x_test, verbose=0)

Epoch 1/100


ValueError: in user code:

    /home/ifeai/anaconda3/envs/py385_gpu/lib/python3.8/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /home/ifeai/anaconda3/envs/py385_gpu/lib/python3.8/site-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/ifeai/anaconda3/envs/py385_gpu/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/ifeai/anaconda3/envs/py385_gpu/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/ifeai/anaconda3/envs/py385_gpu/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/ifeai/anaconda3/envs/py3

In [4]:
# log_dir_lstm_v1 = "logs//hparams//lstm//v1_" + snapshot
# activationfunctions = ['relu', 'sigmoid', 'tanh', 'softmax', 'softsign', 'selu', 'elu']
# def tunner_lstm_model_v1(hp):
#     """Builds a recurrent model."""
#     model = keras.Sequential()
#     model.add(Input(shape=(rows, cols)))
#     model.add(LSTM(units=hp.Int('units_LSTM',
#                                 min_value=2,
#                                 max_value=512,
#                                 step=32), 
#                    activation=hp.Choice('activation_LSTM', activationfunctions)))
#     model.add(Dense(units=hp.Int('units_hidden',
#                                 min_value=2,
#                                 max_value=256,
#                                 step=2),
#                    activation='relu'))
    
#     model.add(Dense(6, activation='softmax'))
#     if (hp.Choice('optimizer', ['adam', 'sgd']) == 'adam'):
#         optimizer = keras.optimizers.Adam(hp.Choice('learning_rate',
#                                   values=[1e-1, 1e-2, 1e-3, 1e-4, 1e-5]))
#     else:
#         optimizer = keras.optimizers.SGD(hp.Choice('learning_rate',
#                                   values=[1e-1, 1e-2, 1e-3, 1e-4, 1e-5]))
#     model.compile(optimizer=optimizer,
#                   loss=tf.keras.losses.CategoricalCrossentropy(),
#                   metrics=[tf.keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
#                            tf.keras.metrics.Precision(name='precision'),
#                            tf.keras.metrics.Recall(name='recall')
#                           ])
    
#     return model

# lstm_tuner_v1=Hyperband(
#     tunner_lstm_model_v1,
#     objective=kt.Objective('val_categorical_accuracy', direction='max'),
#     max_epochs=50,
#     seed=42,
#     directory="logs/tuner/lstm",
#     project_name="MasterThesis",
#     overwrite=True,
#     logger=TensorBoardLogger(
#         metrics=['loss', 'categorical_accuracy', 'val_loss', 'val_categorical_accuracy', 'precision', 'recall'], logdir=log_dir_lstm_v1,
#     ) # add only this argument
# )

# setup_tb(lstm_tuner_v1)  # (Optional) For more accurate visualization.
# lstm_tuner_v1.search(X_train, y_train,
#              epochs=50,
#              batch_size=64,
#              validation_data=(X_val, y_val),
#              shuffle=False
#             )
# tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir_lstm_v1, profile_batch = '500,520')
# bestparams_v1 = lstm_tuner_v1.get_best_hyperparameters(1)[0]
# hyper_model_v1 = lstm_tuner_v1.hypermodel.build(bestparams_v1)
# training_history_v1 = hyper_model_v1.fit(
#     X_train, # input
#     y_train, # output
#     batch_size=64,
#     verbose=0, # Suppress chatty output; use Tensorboard instead
#     epochs=40,
#     validation_data=(X_val, y_val),
#     callbacks=[tensorboard_callback],
# )
# print("Average loss: ", np.average(training_history_v1.history['loss']))
# print("Average val loss: ", np.average(training_history_v1.history['val_loss']))
# print("Average accuracy: ", np.average(training_history_v1.history['categorical_accuracy']))
# print("Average val accuracy: ", np.average(training_history_v1.history['val_categorical_accuracy']))
# train_result = hyper_model_v1.evaluate(X_train, y_train)
# val_result = hyper_model_v1.evaluate(X_val, y_val)
# test_result = hyper_model_v1.evaluate(X_test, y_test)
# newUser_result = hyper_model_v1.evaluate(X_newUser, y_newUser)
# print("Train: ", train_result)
# print("Train: ", val_result)
# print("Train: ", test_result)
# print("Train: ", newUser_result)